## Packages required

In [ ]:
using GridapTopOpt, Gridap, Gridap.TensorValues

In [ ]:
const E = 1        # Young's Modulus      
const ν = 0.3      # Poison ratio    
const G = E/(2*(1+ν))
const l = 8.0      # Bending length scale  
const N = 0.99     # Micropolar parameter
 
# const λₘₐₜ = 2*G*ν/(1 -2*ν)   ## For plain strain
# const κₘₐₜ = 2*G*N^2/(1-N^2)
# const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
# const γₘₐₜ = 4*G*l^2

const λₘₐₜ = E*ν/(1-ν^2)  # Modified λ for plane stress (critical change!)   #2*G*ν/(1 -2*ν)
const κₘₐₜ = 2*G*N^2/(1-N^2)
const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
const γₘₐₜ = 4*G*l^2

In [ ]:
const Height = 10
const Length = 3*Height

In [ ]:
# FE parameters
order = 1                                                            # Finite element order
dom = (0,Length,0,Height)                                            # Bounding domain
nx,ny = (300,100)
el_size = (nx,ny)                                                   # Mesh partition size
f_Γ_N(x) = (x[2] ≈ Height) &&                                       # Γ_N indicator function
    (0.0 <= x[1] <= 5*Length/nx)
f_Γ_D1(x) = (x[1] ≈ 0.0)
f_Γ_D2(x) = (x[1] ≈ Length) && (x[2] ≈ 0.0)

In [ ]:
# FD parameters
γ = 0.1                                                          # HJ eqn time step coeff
γ_reinit = 0.5                                                   # Reinit. eqn time step coeff
max_steps = 45                                                   # Max steps for advection                         
tol =1e-6                                                        # Reinitialisation tolerance

In [ ]:
# Problem parameters
g = VectorValue(0,-1)                                            # Load
vf = 0.3                                                         # Volume fraction constraint
lsf_func = initial_lsf((12/Length),0.2)                          # Initial level set function
iter_mod = 10                                                    # VTK Output modulo                        
path = "./Result_tol&Max_stepMod/results_$nx,$ny/N_$N/ElasticMicropolar_l_$l/$Length,$Height/comp_serial/E_$E/"      # Output path
mkpath(path)                                                     # Create path

In [ ]:
function σ_Bmod(ε)
    σM = ((λₘₐₜ)*tr(ε)*one(ε) + (2*μₘₐₜ + κₘₐₜ)*(ε))
    return σM
end

E_Matrx = TensorValue(0,1,-1,0)

function ε_Skw(∇,θ)
    ∇ᵀ = transpose(∇)
    w = (0.5*(∇ᵀ - ∇)) - (E_Matrx*θ)
    return w
end

function σ_Cmod(ϵ_skew)
    σM = κₘₐₜ*ϵ_skew
    return σM
end

function M_mod(∇)
    M = γₘₐₜ*∇
    return M
end

function Skw(u,θ)
    ∇ᵀ = transpose(∇(u))
    w = (0.5*(∇ᵀ - ∇(u)) - (E_Matrx*θ))
    return w
end

function ElasEnergy(ε_in)
    ElasEner = 0.5*(λₘₐₜ)*(tr(ε_in) * tr(ε_in)) + (μₘₐₜ + 0.5*κₘₐₜ)*(ε_in ⊙ ε_in)
    TotElasEner = sum(∫(ElasEner)*dΩ)
    return TotElasEner
end

function RotationEnergy(∇_th) #∇_th = ∇(θ)
    RotEnergy = 0.5*γₘₐₜ*(∇_th ⋅ ∇_th)
    TotRotEnergy = sum(∫(RotEnergy)*dΩ)
    return TotRotEnergy
end

function CouplingEnergy(∇_uh,θ)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    coupEnergy = 0.5*(κₘₐₜ*(ϵ_skew ⊙ ϵ_skew))
    TotcoupEnergy = sum(∫(coupEnergy)*dΩ)
    return TotcoupEnergy
end

In [ ]:
# Model
model = CartesianDiscreteModel(dom,el_size);
update_labels!(1,model,f_Γ_D1,"Gamma_D1")
update_labels!(2,model,f_Γ_D2,"Gamma_D2")
update_labels!(3,model,f_Γ_N,"Gamma_N")

In [ ]:
writevtk(model,path*"Half_MBBExampleI")

In [ ]:
# Triangulation and measures
Ω = Triangulation(model)
Γ_N = BoundaryTriangulation(model,tags="Gamma_N")
dΩ = Measure(Ω,2*order)
dΓ_N = Measure(Γ_N,2*order)
vol_D = sum(∫(1)dΩ)

In [ ]:
## Spaces
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffe_scalar = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags=["Gamma_D1","Gamma_D2"],dirichlet_masks = [(true,false),(false,true)])
U = TrialFESpace(V,[VectorValue(0.0,0.0),VectorValue(0.0,0.0)])   ## Dispalcement Space
Q = TestFESpace(model,reffe_scalar;conformity=:H1)
P = TrialFESpace(Q)

In [ ]:
UP = MultiFieldFESpace([U,P])
VQ = MultiFieldFESpace([V,Q])

In [ ]:
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar)
U_reg = TrialFESpace(V_reg)

In [ ]:
# Level set and interpolator
φh = interpolate(lsf_func,V_φ)
interp = SmoothErsatzMaterialInterpolation(η = (2)*maximum(get_el_Δ(model)))    # η = 2 ×  maximum side length of an element.
I,H,DH,ρ = interp.I,interp.H,interp.DH,interp.ρ

In [ ]:
writevtk(Ω,path*"initial_lsfHalfMBBI",cellfields=["phi"=>φh,
  "ρ(phi)"=>(ρ ∘ φh),"|nabla(phi)|"=>(norm ∘ ∇(φh))])

In [ ]:
a((u,θ),(w,v),φ) = ∫((I ∘ φ)*((ε(w) ⊙ (σ_Bmod∘(ε(u))) ) + ((Skw(w,v)) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ)))) + ((∇(v))⋅ (M_mod∘(∇(θ)))) - ((v*((E_Matrx) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ))))) )))dΩ;

lm((w,v),φ) = ∫(w·g)dΓ_N


In [ ]:
state_map = RepeatingAffineFEStateMap(1, a, [lm],UP,VQ,V_φ)

In [ ]:
evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
ls_evo = LevelSetEvolution(evo,reinit)

In [ ]:
function Cᴴ(r,s,uϕ,φ,dΩ,dΓ_N)
    u_s = uϕ[2s-1]; θ_s = uϕ[2s];
    ∫((u_s)·g)dΓ_N 
end

J(uϕ,φ) = 1*Cᴴ(1,1,uϕ,φ,dΩ,dΓ_N)
C1(uϕ,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ;

In [ ]:
DC1(q,uϕ,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ

In [ ]:
pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=nothing,analytic_dC=[DC1])

In [ ]:
α = 4max_steps*γ*maximum(get_el_Δ(model))
a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)

In [ ]:
# ## Optimiser
optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;γ,verbose=true,constraint_names=[:Vol])

In [ ]:
for (it,uh,φh) in optimiser
    uv, θv = uh
    data = ["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"θv"=>θv,"ρ(φ)"=>(ρ ∘ φh)]
    iszero(it % iter_mod) && writevtk(Ω,path*"out$it",cellfields= data) 
    write_history(path*"/historymodified$tol,max_steps-$max_steps.txt",optimiser.history)
end

In [ ]:
it = get_history(optimiser).niter; uh = get_state(pcfs);
uv, θv = uh 
writevtk(Ω,path*"out$it,max_step-$max_steps, tol-$tol",cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"θv"=>θv])

In [ ]:
WElas = ElasEnergy(ε(uv))

In [ ]:
WRot = RotationEnergy(∇(θv))

In [ ]:
WElas = CouplingEnergy(∇(uv),θv)